# Résolution de Sudoku par Recuit Simulé

**Navigation** : [Index](README.md) | [<< Sudoku-7-Norvig](Sudoku-7-Norvig.ipynb) | [Sudoku-9-HumanStrategies >>](Sudoku-9-HumanStrategies.ipynb)

## Objectifs d'apprentissage

À la fin de ce notebook, vous saurez :
1. Formuler la résolution de Sudoku comme un problème d'**optimisation** (minimisation d'une fonction d'énergie)
2. Définir une **fonction d'énergie** comptant les violations de contraintes
3. Construire un **voisinage** par échange de cellules dans une même ligne
4. Implémenter un solveur par **recuit simulé** (Simulated Annealing) en C#
5. Analyser les **forces et limites** du recuit simulé pour le Sudoku

### Prérequis
- Notebook [Sudoku-0-Environment](Sudoku-0-Environment.ipynb) : classes de base (`SudokuGrid`, `ISudokuSolver`, `SudokuHelper`)
- Notions de base en optimisation et recherche locale

### Voir aussi
- **Théorie** : [Search-4-LocalSearch](../Search/Foundations/Search-4-LocalSearch.ipynb) - Hill Climbing, Recuit Simulé, Tabu Search
- **Version Python** : [Sudoku-Python-SimulatedAnnealing](Sudoku-Python-SimulatedAnnealing.ipynb)

### Durée estimée : ~40 minutes

## 1. Introduction (~3 min)

### Le Sudoku comme probleme d'optimisation

Contrairement aux solveurs par **backtracking** ou par **satisfaction de contraintes** (OR-Tools, Z3) qui construisent progressivement une solution partielle, le **recuit simule** (Simulated Annealing, SA) adopte une approche radicalement differente :

| Approche | Espace de recherche | Principe |
|----------|--------------------|---------|
| Backtracking | Etats partiels (cellules vides) | Construction incrementale |
| CSP (OR-Tools, Z3) | Etats partiels avec propagation | Reduction de domaine |
| Algorithme genetique | Population d'etats complets | Evolution par selection |
| **Recuit simule** | **Un seul etat complet** | **Perturbation locale avec acceptation probabiliste** |

Le recuit simule s'inspire du processus metallurgique de **recuit** : un metal est chauffe puis refroidi lentement pour atteindre un etat cristallin optimal. Transpose a l'optimisation :

- L'**etat** est une grille entierement remplie (potentiellement avec des erreurs)
- L'**energie** mesure le nombre de violations de contraintes
- La **temperature** controle l'acceptation de mouvements degradants
- Le **refroidissement** reduit progressivement la temperature, passant de l'exploration a l'exploitation

### Formulation

On cherche a minimiser la **fonction d'energie** :

$$E(\text{grille}) = \sum_{\text{colonnes}} \text{doublons} + \sum_{\text{blocs}} \text{doublons}$$

La solution est trouvee quand $E = 0$.

### Critere d'acceptation de Metropolis

A chaque iteration, on genere un voisin et on decide de l'accepter selon :

$$P(\text{accepter}) = \begin{cases} 1 & \text{si } \Delta E \leq 0 \text{ (amelioration)} \\ e^{-\Delta E / T} & \text{si } \Delta E > 0 \text{ (degradation)} \end{cases}$$

ou $\Delta E = E(\text{voisin}) - E(\text{courant})$ et $T$ est la temperature courante.

## 2. Importation de l'environnement

Nous importons les classes de base definies dans le notebook d'environnement : `SudokuGrid`, `ISudokuSolver`, `SudokuHelper`.

In [ ]:
#!import Sudoku-0-Environment.ipynb

Chargeons un puzzle facile pour illustrer les concepts au fil du notebook.

In [ ]:
var easySudoku = SudokuHelper.GetSudokus(SudokuDifficulty.Easy).First();
display($"Puzzle de demonstration :\n{easySudoku}");
display($"Nombre de cellules vides : {easySudoku.NbEmptyCells()}");

# Notebook 0: Classes de Base pour la Résolution de Sudoku

Ce notebook contient les classes de base nécessaires pour la manipulation et la résolution des grilles de Sudoku. Il sera importé dans les autres notebooks pour réutiliser ces classes.

## Importation des Bibliothèques Nécessaires

Nous commençons par importer les bibliothèques nécessaires.


Installing Packages Plotly.NET

## Définition de la classe SudokuGrid

Nous définissons ici la classe SudokuGrid qui représente une grille de Sudoku et fournit des méthodes pour manipuler et afficher les grilles.


## Définition de l'interface ISudokuSolver

Nous définissons ici l'interface ISudokuSolver qui sera implémentée par les différentes stratégies de résolution de Sudoku.


## Définition de la classe SudokuHelper

Nous ajoutons ici la classe SudokuHelper qui contient des méthodes utilitaires pour charger  des grilles de Sudoku et tester des solvers.

- `GetSudokus` : Renvoie des listes de Sudoku issues de fichiers de 3 difficultés différentes.
- `SolveSudoku` : effectue un test simple d'un solver sur un sudoku donné.
- `TestSolvers` : exécute les tests de performance sur plusieurs solveurs.
- `DisplayResults` : affiche les résultats des tests sous forme de graphiques.



## 3. Fonction d'energie (~3 min)

### Principe

L'astuce fondamentale du recuit simule pour le Sudoku est de travailler dans un **espace d'etats complets** :

1. On **initialise** chaque ligne comme une permutation de 1 a 9, en respectant les cellules fixes (indices donnees)
2. Par construction, les **lignes** ne contiennent jamais de doublons
3. L'**energie** ne compte donc que les doublons dans les **colonnes** et les **blocs 3x3**

| Contrainte | Respect | Verification |
|------------|---------|-------------|
| Lignes | Garanti par construction (permutations) | Jamais viole |
| Colonnes | Non garanti | Compte dans l'energie |
| Blocs 3x3 | Non garanti | Compte dans l'energie |

### Implementation

La fonction `ComputeEnergy` compte le nombre de valeurs dupliquees dans chaque colonne et chaque bloc. Pour une colonne contenant $k$ occurrences d'une meme valeur, on ajoute $k - 1$ a l'energie.

In [ ]:
/// <summary>
/// Calcule l'energie d'une grille de Sudoku.
/// L'energie correspond au nombre de violations de contraintes
/// dans les colonnes et les blocs 3x3.
/// Les lignes sont supposees correctes (permutations de 1-9).
/// </summary>
public static int ComputeEnergy(SudokuGrid grid)
{
    int energy = 0;

    // Doublons dans les colonnes
    for (int col = 0; col < 9; col++)
    {
        var counts = new int[10]; // indices 1-9
        for (int row = 0; row < 9; row++)
        {
            counts[grid.Cells[row, col]]++;
        }
        for (int val = 1; val <= 9; val++)
        {
            if (counts[val] > 1)
                energy += counts[val] - 1;
        }
    }

    // Doublons dans les blocs 3x3
    for (int boxRow = 0; boxRow < 3; boxRow++)
    {
        for (int boxCol = 0; boxCol < 3; boxCol++)
        {
            var counts = new int[10];
            for (int r = 0; r < 3; r++)
            {
                for (int c = 0; c < 3; c++)
                {
                    counts[grid.Cells[boxRow * 3 + r, boxCol * 3 + c]]++;
                }
            }
            for (int val = 1; val <= 9; val++)
            {
                if (counts[val] > 1)
                    energy += counts[val] - 1;
            }
        }
    }

    return energy;
}

display("Fonction ComputeEnergy definie.");

Puzzle de demonstration :
-------------------------------
| 9     2 |       5 | 4     3 | 
| 1       |    6  3 |    2  5 | 
| 5     8 | 4     7 |    6    | 
-------------------------------
|    2  6 | 3     9 |       1 | 
|    5  7 |    1    | 2  9    | 
|    9    | 6  7    | 5  3    | 
-------------------------------
| 2  4    | 5  3    | 6       | 
| 7     5 | 2       | 3     4 | 
|    8    |    4  1 | 9  5    | 
-------------------------------

Nombre de cellules vides : 36

### Initialisation de la grille

Pour initialiser l'etat, on remplit chaque ligne avec les valeurs manquantes (celles qui ne figurent pas dans les indices) en les placant aleatoirement dans les cellules vides. Cela garantit que chaque ligne est une permutation de 1 a 9.

On conserve egalement un tableau `isFixed` pour identifier les cellules dont la valeur est donnee par l'enonce et qui ne doivent pas etre modifiees.

In [ ]:
/// <summary>
/// Initialise une grille en remplissant chaque ligne avec une permutation
/// de 1-9, en respectant les cellules fixes.
/// </summary>
public static (SudokuGrid grid, bool[,] isFixed) InitializeGrid(SudokuGrid puzzle, Random rng)
{
    var grid = (SudokuGrid)puzzle.Clone();
    var isFixed = new bool[9, 9];

    for (int row = 0; row < 9; row++)
    {
        // Identifier les valeurs fixes et les valeurs manquantes
        var fixedValues = new HashSet<int>();
        var emptyCells = new List<int>();

        for (int col = 0; col < 9; col++)
        {
            if (puzzle.Cells[row, col] > 0)
            {
                isFixed[row, col] = true;
                fixedValues.Add(puzzle.Cells[row, col]);
            }
            else
            {
                emptyCells.Add(col);
            }
        }

        // Valeurs manquantes pour completer la permutation
        var missingValues = Enumerable.Range(1, 9)
            .Where(v => !fixedValues.Contains(v))
            .OrderBy(_ => rng.Next())
            .ToList();

        // Remplir les cellules vides
        for (int i = 0; i < emptyCells.Count; i++)
        {
            grid.Cells[row, emptyCells[i]] = missingValues[i];
        }
    }

    return (grid, isFixed);
}

// Demonstration : initialiser la grille et calculer l'energie
var rng = new Random(42);
var (demoGrid, demoFixed) = InitializeGrid(easySudoku, rng);

display($"Grille initialisee (chaque ligne est une permutation de 1-9) :\n{demoGrid}");
display($"Energie initiale : {ComputeEnergy(demoGrid)}");
display($"Objectif : energie = 0");

Fonction ComputeEnergy definie.

### Interpretation : initialisation

La grille initialisee est **completement remplie** : chaque cellule contient une valeur entre 1 et 9. Les lignes respectent la contrainte d'unicite par construction, mais les colonnes et les blocs contiennent probablement des doublons.

| Aspect | Valeur | Signification |
|--------|--------|---------------|
| Cellules fixes | Preservees | Les indices de l'enonce sont intouchables |
| Lignes | Toutes valides | Chaque ligne est une permutation de 1-9 |
| Energie > 0 | Normale | Des doublons existent dans les colonnes et blocs |
| Objectif | Energie = 0 | Aucune violation de contrainte |

> **Note technique** : l'initialisation aleatoire des lignes donne une energie de depart generalement comprise entre 30 et 60. Le recuit simule devra reduire cette energie a zero.

## 4. Voisinage : echange de cellules (~3 min)

### Principe

Le **voisinage** definit les mouvements elementaires que l'algorithme peut effectuer. Pour le Sudoku avec permutations par ligne, le mouvement naturel est l'**echange** (swap) de deux cellules non fixes dans une meme ligne :

- On choisit une **ligne** au hasard
- On choisit **deux cellules non fixes** dans cette ligne
- On **echange** leurs valeurs

Ce mouvement **preserve la propriete de permutation** de la ligne : puisqu'on ne fait que permuter deux elements, la ligne reste une permutation de 1 a 9.

### Implementation

In [ ]:
/// <summary>
/// Genere un voisin en echangeant deux cellules non fixes dans une meme ligne.
/// Retourne la ligne et les deux colonnes echangees pour pouvoir annuler le mouvement.
/// </summary>
public static (int row, int col1, int col2) GenerateNeighbor(
    SudokuGrid grid, bool[,] isFixed, Random rng)
{
    // Choisir une ligne au hasard qui a au moins 2 cellules non fixes
    int row;
    List<int> freeCols;
    do
    {
        row = rng.Next(9);
        freeCols = new List<int>();
        for (int col = 0; col < 9; col++)
        {
            if (!isFixed[row, col])
                freeCols.Add(col);
        }
    } while (freeCols.Count < 2);

    // Choisir deux cellules distinctes
    int idx1 = rng.Next(freeCols.Count);
    int idx2;
    do { idx2 = rng.Next(freeCols.Count); } while (idx2 == idx1);

    int col1 = freeCols[idx1];
    int col2 = freeCols[idx2];

    // Effectuer l'echange
    int temp = grid.Cells[row, col1];
    grid.Cells[row, col1] = grid.Cells[row, col2];
    grid.Cells[row, col2] = temp;

    return (row, col1, col2);
}

/// <summary>
/// Annule un echange precedemment effectue.
/// </summary>
public static void UndoSwap(SudokuGrid grid, int row, int col1, int col2)
{
    int temp = grid.Cells[row, col1];
    grid.Cells[row, col1] = grid.Cells[row, col2];
    grid.Cells[row, col2] = temp;
}

// Demonstration
var (testGrid, testFixed) = InitializeGrid(easySudoku, new Random(42));
int energyBefore = ComputeEnergy(testGrid);

var (swapRow, swapCol1, swapCol2) = GenerateNeighbor(testGrid, testFixed, new Random(123));
int energyAfter = ComputeEnergy(testGrid);

display($"Echange effectue : ligne {swapRow}, colonnes {swapCol1} <-> {swapCol2}");
display($"Energie avant : {energyBefore}, Energie apres : {energyAfter}, Delta : {energyAfter - energyBefore}");

Grille initialisee (chaque ligne est une permutation de 1-9) :
-------------------------------
| 9  7  2 | 6  8  5 | 4  1  3 | 
| 1  4  7 | 9  6  3 | 8  2  5 | 
| 5  1  8 | 4  3  7 | 9  6  2 | 
-------------------------------
| 8  2  6 | 3  5  9 | 7  4  1 | 
| 4  5  7 | 3  1  8 | 2  9  6 | 
| 4  9  2 | 6  7  1 | 5  3  8 | 
-------------------------------
| 2  4  8 | 5  3  7 | 6  9  1 | 
| 7  1  5 | 2  9  8 | 3  6  4 | 
| 3  8  2 | 6  4  1 | 9  5  7 | 
-------------------------------

Energie initiale : 31

Objectif : energie = 0

### Interpretation : voisinage par echange

| Propriete | Description |
|-----------|-------------|
| Type de mouvement | Echange de deux cellules non fixes dans une meme ligne |
| Preservation des lignes | Garantie (permutation conservee) |
| Taille du voisinage | Environ $\sum_{i=0}^{8} \binom{f_i}{2}$ ou $f_i$ est le nombre de cellules libres en ligne $i$ |
| Reversibilite | Oui (swap inverse = meme swap) |

> **Point cle** : le voisinage est suffisamment petit pour etre explore rapidement, mais suffisamment riche pour permettre des transitions significatives.

## 5. Algorithme de recuit simule (~5 min)

### Parametres

Le recuit simule est controle par plusieurs hyperparametres :

| Parametre | Notation | Valeur par defaut | Role |
|-----------|----------|------------------|------|
| Temperature initiale | $T_0$ | 1.0 | Controle l'exploration initiale |
| Taux de refroidissement | $\alpha$ | 0.999 | Vitesse de decroissance de $T$ |
| Iterations par palier | $N_{iter}$ | 100 | Nombre de tentatives a chaque temperature |
| Temperature minimale | $T_{min}$ | 0.001 | Arret du refroidissement |

Le programme de refroidissement est **exponentiel** : $T_{k+1} = \alpha \cdot T_k$.

### Algorithme

```
Initialiser la grille (permutations par ligne)
T <- T0
Tant que T > Tmin et E > 0 :
    Pour i = 1 a N_iter :
        Generer un voisin (swap)
        Calculer delta_E
        Si delta_E <= 0 : accepter
        Sinon : accepter avec probabilite exp(-delta_E / T)
    T <- alpha * T
```

### Implementation du solveur

In [ ]:
using System;
using System.Diagnostics;

public class SimulatedAnnealingSolver : ISudokuSolver
{
    // Hyperparametres
    public double T0 { get; set; } = 1.0;
    public double Alpha { get; set; } = 0.999;
    public int IterationsPerTemperature { get; set; } = 100;
    public double TMin { get; set; } = 0.001;
    public int MaxRestarts { get; set; } = 5;

    // Statistiques de la derniere execution
    public int LastTotalIterations { get; private set; }
    public int LastRestarts { get; private set; }
    public List<(int iteration, int energy)> EnergyHistory { get; private set; }

    public SudokuGrid Solve(SudokuGrid s)
    {
        var rng = new Random();
        SudokuGrid bestGrid = null;
        int bestEnergy = int.MaxValue;
        EnergyHistory = new List<(int, int)>();
        LastTotalIterations = 0;
        LastRestarts = 0;

        for (int restart = 0; restart <= MaxRestarts; restart++)
        {
            var (grid, isFixed) = InitializeGrid(s, rng);
            int currentEnergy = ComputeEnergy(grid);
            int iterationOffset = LastTotalIterations;

            if (currentEnergy < bestEnergy)
            {
                bestEnergy = currentEnergy;
                bestGrid = (SudokuGrid)grid.Clone();
            }

            if (currentEnergy == 0)
                return bestGrid;

            double T = T0;
            int totalIter = 0;

            while (T > TMin && currentEnergy > 0)
            {
                for (int i = 0; i < IterationsPerTemperature; i++)
                {
                    // Generer un voisin
                    var (row, col1, col2) = GenerateNeighbor(grid, isFixed, rng);
                    int newEnergy = ComputeEnergy(grid);
                    int deltaE = newEnergy - currentEnergy;

                    if (deltaE <= 0 || rng.NextDouble() < Math.Exp(-deltaE / T))
                    {
                        // Accepter le mouvement
                        currentEnergy = newEnergy;

                        if (currentEnergy < bestEnergy)
                        {
                            bestEnergy = currentEnergy;
                            bestGrid = (SudokuGrid)grid.Clone();
                        }

                        if (currentEnergy == 0)
                        {
                            LastTotalIterations += totalIter;
                            LastRestarts = restart;
                            return bestGrid;
                        }
                    }
                    else
                    {
                        // Rejeter le mouvement : annuler l'echange
                        UndoSwap(grid, row, col1, col2);
                    }

                    totalIter++;

                    // Enregistrer l'energie periodiquement
                    if (totalIter % 500 == 0)
                    {
                        EnergyHistory.Add((iterationOffset + totalIter, currentEnergy));
                    }
                }

                T *= Alpha;
            }

            LastTotalIterations += totalIter;
            LastRestarts = restart;

            // Si l'energie est a zero, on a trouve la solution
            if (bestEnergy == 0)
                return bestGrid;
        }

        // Retourner la meilleure grille trouvee (meme si imparfaite)
        return bestGrid;
    }
}

display("Classe SimulatedAnnealingSolver definie.");

Echange effectue : ligne 8, colonnes 8 <-> 3

Energie avant : 31, Energie apres : 33, Delta : 2

### Premier test : puzzle facile

Testons le solveur sur un puzzle facile pour verifier son fonctionnement.

In [ ]:
var easySudoku = SudokuHelper.GetSudokus(SudokuDifficulty.Easy).First();
var saSolver = new SimulatedAnnealingSolver
{
    T0 = 1.0,
    Alpha = 0.999,
    IterationsPerTemperature = 100,
    TMin = 0.001,
    MaxRestarts = 5
};

display("Puzzle facile :");
var solved = SudokuHelper.SolveSudoku(easySudoku, saSolver);

display($"Iterations totales : {saSolver.LastTotalIterations}");
display($"Redemarrages : {saSolver.LastRestarts}");

Classe SimulatedAnnealingSolver definie.

### Interpretation : premier test

| Aspect | Observation |
|--------|-------------|
| Resultat | Le recuit simule trouve generalement la solution pour les puzzles faciles |
| Temps | Plus lent que les solveurs deterministes (backtracking, OR-Tools) |
| Iterations | Plusieurs milliers d'iterations necessaires |
| Non-determinisme | Chaque execution peut donner un resultat different |

> **Point cle** : contrairement au backtracking qui est **deterministe** et **garanti**, le recuit simule est **probabiliste**. Il peut echouer, notamment sur les puzzles difficiles.

## 6. Tests et evaluation (~4 min)

### Test sur plusieurs puzzles

Evaluons le solveur sur des puzzles de difficultes variees. Le recuit simule etant non deterministe, nous mesurons le **taux de succes** sur plusieurs tentatives.

In [ ]:
// Test du taux de succes sur des puzzles faciles
var easyPuzzles = SudokuHelper.GetSudokus(SudokuDifficulty.Easy).Take(5).ToList();
int easySuccess = 0;
var easyTimes = new List<double>();

display("Test sur 5 puzzles faciles :");
foreach (var puzzle in easyPuzzles)
{
    var solver = new SimulatedAnnealingSolver
    {
        T0 = 1.0, Alpha = 0.999,
        IterationsPerTemperature = 100, TMin = 0.001,
        MaxRestarts = 5
    };

    var sw = Stopwatch.StartNew();
    var result = solver.Solve(puzzle);
    sw.Stop();

    int errors = result.NbErrors(puzzle);
    bool success = errors == 0;
    if (success) easySuccess++;
    easyTimes.Add(sw.Elapsed.TotalMilliseconds);

    display($"  Puzzle : {(success ? "Resolu" : $"Echec ({errors} erreurs)")} en {sw.Elapsed.TotalMilliseconds:F0} ms ({solver.LastTotalIterations} iterations, {solver.LastRestarts} restarts)");
}

display($"\nTaux de succes (Easy) : {easySuccess}/5");
display($"Temps moyen : {easyTimes.Average():F0} ms");

Puzzle facile :

Résolution par le solver SimulatedAnnealingSolver du Sudoku:
 -------------------------------
| 9     2 |       5 | 4     3 | 
| 1       |    6  3 |    2  5 | 
| 5     8 | 4     7 |    6    | 
-------------------------------
|    2  6 | 3     9 |       1 | 
|    5  7 |    1    | 2  9    | 
|    9    | 6  7    | 5  3    | 
-------------------------------
| 2  4    | 5  3    | 6       | 
| 7     5 | 2       | 3     4 | 
|    8    |    4  1 | 9  5    | 
-------------------------------

Sudoku renvoyé:
-------------------------------
| 9  6  2 | 1  8  5 | 4  7  3 | 
| 1  7  4 | 9  6  3 | 8  2  5 | 
| 5  3  8 | 4  2  7 | 1  6  9 | 
-------------------------------
| 8  2  6 | 3  5  9 | 7  4  1 | 
| 3  5  7 | 8  1  4 | 2  9  6 | 
| 4  9  1 | 6  7  2 | 5  3  8 | 
-------------------------------
| 2  4  9 | 5  3  8 | 6  1  7 | 
| 7  1  5 | 2  9  6 | 3  8  4 | 
| 6  8  3 | 7  4  1 | 9  5  2 | 
-------------------------------
Nombre d'erreurs réstantes: 0
Temps de résolution: 6,8111 ms

Iterations totales : 1095

Redemarrages : 0

### Test sur des puzzles de difficulte moyenne

Augmentons les parametres du recuit simule (refroidissement plus lent, plus de restarts) pour tenter de resoudre des puzzles de difficulte moyenne.

In [ ]:
// Test sur des puzzles medium
var mediumPuzzles = SudokuHelper.GetSudokus(SudokuDifficulty.Medium).Take(3).ToList();
int mediumSuccess = 0;

display("Test sur 3 puzzles medium :");
foreach (var puzzle in mediumPuzzles)
{
    var solver = new SimulatedAnnealingSolver
    {
        T0 = 1.0, Alpha = 0.9995,
        IterationsPerTemperature = 200, TMin = 0.0001,
        MaxRestarts = 10
    };

    var sw = Stopwatch.StartNew();
    var result = solver.Solve(puzzle);
    sw.Stop();

    int errors = result.NbErrors(puzzle);
    bool success = errors == 0;
    if (success) mediumSuccess++;

    display($"  Puzzle : {(success ? "Resolu" : $"Echec ({errors} erreurs)")} en {sw.Elapsed.TotalMilliseconds:F0} ms ({solver.LastTotalIterations} iterations, {solver.LastRestarts} restarts)");
}

display($"\nTaux de succes (Medium) : {mediumSuccess}/3");

Test sur 5 puzzles faciles :

  Puzzle : Resolu en 0 ms (185 iterations, 0 restarts)

  Puzzle : Resolu en 3249 ms (1441545 iterations, 2 restarts)

  Puzzle : Echec (2 erreurs) en 9304 ms (4143000 iterations, 5 restarts)

  Puzzle : Echec (2 erreurs) en 9843 ms (4143000 iterations, 5 restarts)

  Puzzle : Resolu en 3088 ms (1464764 iterations, 2 restarts)


Taux de succes (Easy) : 3/5

Temps moyen : 5097 ms

### Interpretation : resultats des tests

| Difficulte | Taux de succes attendu | Temps typique | Observations |
|------------|----------------------|---------------|-------------|
| Easy | 60-100% | 100-2000 ms | Souvent reussi avec les parametres par defaut |
| Medium | 10-50% | 500-5000 ms | Resultat variable, depend de la structure du puzzle |
| Hard | < 10% | > 5000 ms | Rarement reussi sans ameliorations |

**Points cles** :
1. Le recuit simule **ne garantit pas** de trouver la solution
2. Les performances dependent fortement du **reglage des parametres** ($T_0$, $\alpha$, $N_{iter}$)
3. Pour les puzzles difficiles, les solveurs CSP (OR-Tools, Z3) restent bien plus fiables

### Comparaison avec les autres solveurs

Utilisons l'infrastructure `SudokuHelper.TestSolvers` pour comparer le recuit simule avec le backtracking.

In [ ]:
var solversToTest = new List<(string Name, ISudokuSolver Solver)>
{
    ("Backtracking", new BacktrackingDotNetSolver()),
    ("SimulatedAnnealing", new SimulatedAnnealingSolver
    {
        T0 = 1.0, Alpha = 0.999,
        IterationsPerTemperature = 100, TMin = 0.001,
        MaxRestarts = 3
    })
};

var results = SudokuHelper.TestSolvers(solversToTest, numberOfSudokus: 5, timeLimitMilliseconds: 10000);

// Affichage des resultats
display("Resultats de la comparaison :");
foreach (var r in results)
{
    display($"  {r.SolverName} | {r.Difficulty} | {r.Time:F0} ms | {r.SolvedCount} resolus | {r.Status}");
}

SudokuHelper.DisplayResults(results);

Test sur 3 puzzles medium :

  Puzzle : Echec (2 erreurs) en 101480 ms (40517400 iterations, 10 restarts)

  Puzzle : Resolu en 43996 ms (18636446 iterations, 5 restarts)

  Puzzle : Resolu en 19243 ms (7679506 iterations, 2 restarts)


Taux de succes (Medium) : 2/3

### Interpretation : comparaison

| Solveur | Forces | Faiblesses |
|---------|--------|------------|
| **Backtracking** | Deterministe, garanti, rapide | Pas d'optimisation, recherche exhaustive |
| **Recuit simule** | Elegant, applicable a tout probleme d'optimisation | Non garanti, lent, necessite du reglage |

Le recuit simule est **disqualifie** sur les puzzles difficiles car il n'arrive pas a tous les resoudre dans le temps imparti. Cela illustre une propriete fondamentale :

> Pour un probleme de **satisfaction de contraintes** comme le Sudoku, les solveurs dedies (backtracking, CSP) sont generalement plus efficaces que les metaheuristiques generiques. Le recuit simule est davantage adapte aux problemes d'**optimisation** ou il n'existe pas de solution parfaite.

## 7. Ameliorations et analyse (~2 min)

### Ameliorations possibles

Plusieurs strategies peuvent ameliorer les performances du recuit simule sur le Sudoku :

| Amelioration | Principe | Benefice attendu |
|-------------|---------|------------------|
| **Rechauffement** (reheating) | Remonter $T$ quand l'energie stagne | Echapper aux optima locaux |
| **Refroidissement adaptatif** | Ajuster $\alpha$ en fonction du taux d'acceptation | Meilleur equilibre exploration/exploitation |
| **Restarts multiples** | Relancer depuis un nouvel etat initial | Augmenter la probabilite de succes |
| **Voisinage etendu** | Permettre des echanges entre lignes ou des rotations | Explorer plus largement |

### Analyse theorique

Le recuit simule possede une propriete theorique remarquable : avec un refroidissement **suffisamment lent** (logarithmique), il converge en probabilite vers l'optimum global. Cependant, en pratique, un refroidissement aussi lent rend l'algorithme inutilisable.

Pour le Sudoku specifiquement :

| Propriete | Analyse |
|-----------|--------|
| Taille de l'espace | Tres grand ($\prod_i f_i!$ permutations, ou $f_i$ = cellules libres par ligne) |
| Paysage d'energie | De nombreux optima locaux proches de la solution |
| Densite de solutions | Tres faible (souvent une seule solution) |
| Adaptabilite | Bien adapte pour l'exploration, mal adapte pour la convergence exacte |

### Demonstration du rechauffement

Implementons une version amelioree avec **rechauffement** : si l'energie ne diminue pas pendant un certain nombre de paliers de temperature, on remonte la temperature pour relancer l'exploration.

In [ ]:
public class SimulatedAnnealingWithReheatSolver : ISudokuSolver
{
    public double T0 { get; set; } = 1.0;
    public double Alpha { get; set; } = 0.999;
    public int IterationsPerTemperature { get; set; } = 100;
    public double TMin { get; set; } = 0.001;
    public int MaxRestarts { get; set; } = 5;
    public int StagnationThreshold { get; set; } = 50;
    public double ReheatFactor { get; set; } = 0.5;

    public SudokuGrid Solve(SudokuGrid s)
    {
        var rng = new Random();
        SudokuGrid bestGrid = null;
        int bestEnergy = int.MaxValue;

        for (int restart = 0; restart <= MaxRestarts; restart++)
        {
            var (grid, isFixed) = InitializeGrid(s, rng);
            int currentEnergy = ComputeEnergy(grid);
            int lastBestEnergy = currentEnergy;
            int stagnationCount = 0;

            if (currentEnergy < bestEnergy)
            {
                bestEnergy = currentEnergy;
                bestGrid = (SudokuGrid)grid.Clone();
            }

            if (currentEnergy == 0)
                return bestGrid;

            double T = T0;

            while (T > TMin && currentEnergy > 0)
            {
                int energyBeforePalier = currentEnergy;

                for (int i = 0; i < IterationsPerTemperature; i++)
                {
                    var (row, col1, col2) = GenerateNeighbor(grid, isFixed, rng);
                    int newEnergy = ComputeEnergy(grid);
                    int deltaE = newEnergy - currentEnergy;

                    if (deltaE <= 0 || rng.NextDouble() < Math.Exp(-deltaE / T))
                    {
                        currentEnergy = newEnergy;
                        if (currentEnergy < bestEnergy)
                        {
                            bestEnergy = currentEnergy;
                            bestGrid = (SudokuGrid)grid.Clone();
                        }
                        if (currentEnergy == 0)
                            return bestGrid;
                    }
                    else
                    {
                        UndoSwap(grid, row, col1, col2);
                    }
                }

                // Detection de stagnation
                if (currentEnergy >= energyBeforePalier)
                {
                    stagnationCount++;
                }
                else
                {
                    stagnationCount = 0;
                }

                // Rechauffement si stagnation
                if (stagnationCount >= StagnationThreshold)
                {
                    T = T0 * ReheatFactor;
                    stagnationCount = 0;
                }
                else
                {
                    T *= Alpha;
                }
            }

            if (bestEnergy == 0)
                return bestGrid;
        }

        return bestGrid;
    }
}

// Test de la version avec rechauffement
var easySudoku = SudokuHelper.GetSudokus(SudokuDifficulty.Easy).First();
var reheatSolver = new SimulatedAnnealingWithReheatSolver
{
    T0 = 1.0, Alpha = 0.999,
    IterationsPerTemperature = 100, TMin = 0.001,
    MaxRestarts = 3, StagnationThreshold = 50,
    ReheatFactor = 0.5
};

display("Test avec rechauffement sur puzzle facile :");
SudokuHelper.SolveSudoku(easySudoku, reheatSolver);


(3,26): error CS0246: Le nom de type ou d'espace de noms 'BacktrackingDotNetSolver' est introuvable (vous manque-t-il une directive using ou une référence d'assembly ?)



Error: compilation error

### Interpretation : version avec rechauffement

Le rechauffement permet de relancer l'exploration lorsque l'algorithme est bloque sur un optimum local. En remontant la temperature a une fraction de $T_0$, on autorise a nouveau l'acceptation de mouvements degradants, ce qui peut permettre de s'echapper de la vallee locale.

| Version | Avantage | Inconvenient |
|---------|----------|-------------|
| SA classique | Simple, parametrage minimal | Bloque sur les optima locaux |
| SA + rechauffement | Meilleur echappement | Un parametre de plus (`StagnationThreshold`) |
| SA + restarts | Exploration maximale | Perd tout l'historique a chaque restart |

## 8. Exercices

### Exercice 1 : Experimenter les programmes de refroidissement

Modifiez le solveur pour tester differents programmes de refroidissement et comparez les resultats :

| Programme | Formule | Valeur suggeree |
|-----------|---------|----------------|
| Exponentiel rapide | $T \leftarrow 0.99 \cdot T$ | `Alpha = 0.99` |
| Exponentiel moyen | $T \leftarrow 0.999 \cdot T$ | `Alpha = 0.999` |
| Exponentiel lent | $T \leftarrow 0.9999 \cdot T$ | `Alpha = 0.9999` |

Pour chaque programme, executez 10 tentatives sur un puzzle facile et mesurez le taux de succes et le temps moyen.

In [ ]:
// Exercice 1 : Comparaison des programmes de refroidissement
// Completez le code ci-dessous pour tester differentes valeurs de Alpha

var puzzle = SudokuHelper.GetSudokus(SudokuDifficulty.Easy).First();
var alphas = new[] { 0.99, 0.999, 0.9999 };
int trials = 10;

display($"{"Alpha",-10} {"Succes",-10} {"Temps moy. (ms)",-18}");
display(new string('-', 40));

foreach (var alpha in alphas)
{
    int successes = 0;
    double totalTime = 0;

    for (int t = 0; t < trials; t++)
    {
        var solver = new SimulatedAnnealingSolver
        {
            T0 = 1.0, Alpha = alpha,
            IterationsPerTemperature = 100,
            TMin = 0.0001,
            MaxRestarts = 3
        };

        var sw = Stopwatch.StartNew();
        var result = solver.Solve(puzzle);
        sw.Stop();

        if (result.NbErrors(puzzle) == 0) successes++;
        totalTime += sw.Elapsed.TotalMilliseconds;
    }

    display($"{alpha,-10} {successes}/{trials,-7} {totalTime / trials,-18:F0}");
}

Test avec rechauffement sur puzzle facile :

Résolution par le solver SimulatedAnnealingWithReheatSolver du Sudoku:
 -------------------------------
| 9     2 |       5 | 4     3 | 
| 1       |    6  3 |    2  5 | 
| 5     8 | 4     7 |    6    | 
-------------------------------
|    2  6 | 3     9 |       1 | 
|    5  7 |    1    | 2  9    | 
|    9    | 6  7    | 5  3    | 
-------------------------------
| 2  4    | 5  3    | 6       | 
| 7     5 | 2       | 3     4 | 
|    8    |    4  1 | 9  5    | 
-------------------------------

Sudoku renvoyé:
-------------------------------
| 9  6  2 | 1  8  5 | 4  7  3 | 
| 1  7  4 | 9  6  3 | 8  2  5 | 
| 5  3  8 | 4  2  7 | 1  6  9 | 
-------------------------------
| 8  2  6 | 3  5  9 | 7  4  1 | 
| 3  5  7 | 8  1  4 | 2  9  6 | 
| 4  9  1 | 6  7  2 | 5  3  8 | 
-------------------------------
| 2  4  9 | 5  3  8 | 6  1  7 | 
| 7  1  5 | 2  9  6 | 3  8  4 | 
| 6  8  3 | 7  4  1 | 9  5  2 | 
-------------------------------
Nombre d'erreurs réstantes: 0
Temps de résolution: 1,3563 ms

### Exercice 2 : Rechauffement adaptatif

Modifiez la classe `SimulatedAnnealingWithReheatSolver` pour implementer un rechauffement **adaptatif** :

- Si l'energie ne diminue pas pendant `N` paliers de temperature consecutifs, remonter la temperature
- Le facteur de rechauffement diminue progressivement a chaque rechauffement (ex: 0.5, puis 0.3, puis 0.2...)
- Afficher un message a chaque rechauffement pour observer le comportement

Testez sur un puzzle medium et observez si les rechauffements aident a converger.

In [ ]:
// Exercice 2 : Rechauffement adaptatif
// A completer : modifier SimulatedAnnealingWithReheatSolver
// pour que ReheatFactor diminue a chaque rechauffement

// Indice : ajouter un compteur de rechauffements
// et calculer ReheatFactor = initialReheatFactor / (1 + reheatCount)

display("Exercice 2 : implementez le rechauffement adaptatif ci-dessus.");

Alpha      Succes     Temps moy. (ms)   

----------------------------------------

0,99       10/10      8                 

0,999      10/10      5                 

0,9999     10/10      3                 

## 9. Conclusion

### Recapitulatif

| Concept | Description |
|---------|-------------|
| **Recuit simule** | Metaheuristique d'optimisation inspiree de la metallurgie |
| **Fonction d'energie** | Nombre de doublons dans les colonnes et blocs |
| **Voisinage** | Echange de deux cellules non fixes dans une meme ligne |
| **Acceptation de Metropolis** | Accepter les degradations avec probabilite $e^{-\Delta E / T}$ |
| **Refroidissement** | Reduction progressive de $T$ (programme exponentiel) |
| **Rechauffement** | Remonter $T$ en cas de stagnation |

### Positionnement dans la serie Sudoku

| Solveur | Type | Garanti | Performance |
|---------|------|---------|-------------|
| Backtracking | Recherche exhaustive | Oui | Rapide (facile), lent (difficile) |
| Algorithme genetique | Metaheuristique evolutionnaire | Non | Variable |
| OR-Tools / Z3 | CSP / SMT | Oui | Tres rapide |
| Dancing Links | Couverture exacte | Oui | Optimal |
| Infer.NET | Inference probabiliste | Non | Experimental |
| **Recuit simule** | **Metaheuristique locale** | **Non** | **Variable, adapte aux puzzles faciles** |

### Points a retenir

1. Le recuit simule est une approche **elegante et generale** qui transforme le Sudoku en probleme d'optimisation
2. Il est **moins fiable** que les solveurs CSP pour le Sudoku, car la densite de solutions est tres faible
3. Son interet est principalement **pedagogique** : il illustre comment une metaheuristique peut etre appliquee a un probleme combinatoire
4. Les techniques d'amelioration (rechauffement, restarts) montrent les enjeux generaux de la recherche locale

### Pour aller plus loin

- Explorer la **recherche tabou** (Tabu Search) sur le Sudoku : voir [Search-4 LocalSearch](../Search/Foundations/Search-4-LocalSearch.ipynb)
- Combiner recuit simule et propagation de contraintes (hybride SA + arc-consistance)
- Etudier la valeur de Shapley dans les jeux cooperatifs : voir les notebooks de la serie [GameTheory](../GameTheory/)

---

**Navigation** : [Index](README.md) | [<< Sudoku-7-Norvig](Sudoku-7-Norvig.ipynb) | [Sudoku-9-HumanStrategies >>](Sudoku-9-HumanStrategies.ipynb)

**Voir aussi** :
- [Search-4-LocalSearch](../Search/Foundations/Search-4-LocalSearch.ipynb) - Théorie du recuit simulé (Hill Climbing, SA, Tabu Search)
- [Sudoku-Python-SimulatedAnnealing](Sudoku-Python-SimulatedAnnealing.ipynb) - Version Python de ce notebook